## 从key文件读取key值
*输入参数: file_path
文件路径
*返回值: 存有key的列表

In [1]:
def read_keys_from_file(key_path):
    key_pool = []
    with open(key_path, 'r') as file:
        for line in file:
            key = line.strip()  # 去掉每一行的换行符
            key_pool.append(key)  # 将每一行存入key池中
    return key_pool

In [2]:
# 读取key
key_path = "key.txt"  # 将这个替换为你的txt文件的路径
key_pool = read_keys_from_file(key_path)

In [3]:
import os
from random import random
os.environ['HTTP_PROXY'] = "127.0.0.1:33210"
os.environ['HTTPS_PROXY'] = "127.0.0.1:33210"
os.environ["OPENAI_API_KEY"] = key_pool[1]

from langchain.document_loaders import PyPDFLoader

from PyPDF2 import PdfMerger
file_list = ["KT820说明书V2.0 201909.pdf","GSK980TC3系列  编程操作手册（2018年6月第3版）0622定(1).pdf"]
file_merger = PdfMerger()       # 合并pdf文件
for pdf in file_list:
    file_merger.append(pdf)
# 合并后的pdf放在本地
file_merger.write("merge.pdf")
loader = PyPDFLoader("merge.pdf")
unprocessed_documents = loader.load()


In [245]:
# 用户输入的问题
question = "kt820"

import jieba
# 利用jieba_fast进行分词
def jieba_extract_keywords(question):
    # 使用jieba进行分词
    keywords = jieba.cut(question)
    return keywords

# 利用chatgpt进行分词
def gpt_extract_keywords(question):

    from langchain.prompts import ChatPromptTemplate
    prompts = f"""
        ----
        {question}
        ----
        第一步.对question进行分词。注意如果question是数学表达式或包含数学表达式，不需要提取数学表达式中的关键词而是直接返回完整的数学表达式。
        第二步.你是一个专家级的文本关键词提取大师，根据第一步的分词后的结果，提取其中关键词并按行地返回。关键词提取应该尽可能全面且详细，包括单个有意义的词汇和词组。
        只需要输出关键词，只需要输出关键词！
        注意按行返回关键词！
        如果提取关键词为空，直接返回{question}！

    """


    promptsTemplate = ChatPromptTemplate.from_template(prompts)
    from langchain.chains import LLMChain
    from langchain.chat_models import ChatOpenAI
    llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=0)
    chain = LLMChain(llm=llm, prompt=promptsTemplate)
    response = chain.run(question=question)
    return response

In [246]:
import PyPDF2
# 在pdf中搜索关键词并且返回页面
def search_keywords_in_pdf(doc,keywords):
    keyword_count_per_page = {}
    for page_num in range(len(doc)):
        # 计算当前页面的关键词出现次数
        count = sum(doc[page_num].page_content.lower().count(keyword.lower()) for keyword in keywords)

        # 如果关键词出现次数大于0，则记录该页码和次数
        if count > 0:
            keyword_count_per_page[page_num+1] = count
    # 根据关键词出现次数从大到小排序页面号
    sorted_page_numbers = [k for k, v in sorted(keyword_count_per_page.items(), key=lambda item: item[1],reverse=True)]
    return sorted_page_numbers

def get_page_content(file_path, page_number):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        # 确保页面号在有效范围内
        if page_number - 1 < len(reader.pages):
            page = reader.pages[page_number - 1]
            text = page.extract_text()
            return text
        else:
            return "指定的页码超出了PDF文件的范围"


In [247]:
import time
import jieba.analyse as analyse
start_time = time.time()        # 记录开始时间

# 从问题中提取关键词
# 参数说明 :
# sentence 需要提取的字符串，必须是str类型，不能是list
# topK 提取前多少个关键字
# withWeight 是否返回每个关键词的权重
# allowPOS是允许的提取的词性，默认为allowPOS=‘ns’, ‘n’, ‘vn’, ‘v’，提取地名、名词、动名词、动词
keywords = jieba.analyse.extract_tags(question, topK=6, withWeight=False, allowPOS=())
non_keywords = jieba.analyse.extract_tags(question, topK=6, withWeight=False, allowPOS=('v','vd','vn','a','ad','an','d','w','c','u','r','p'))
print('非关键词：',non_keywords)
keywords = list(set(keywords)-set(non_keywords))    # 取差集


print('关键词：',keywords)
print(type(keywords))

非关键词： []
关键词： ['kt820']
<class 'list'>


In [248]:

os.environ["OPENAI_API_KEY"] = key_pool[2]
from langchain.chat_models import ChatOpenAI


In [249]:
pages = search_keywords_in_pdf(unprocessed_documents,keywords)
print(f"关键词匹配的页面有: {pages}")

关键词匹配的页面有: []


In [253]:
documents = []
# 控制页面的数量
pages = pages[:30]

from collections import defaultdict
# 使用 defaultdict 存储未处理的文档，键为页面编号，值为文档列表
unprocessed_documents_dict = defaultdict(list)
for doc in unprocessed_documents:
    unprocessed_documents_dict[doc.metadata['page']].append(doc)

# 使用列表推导式来按页面顺序添加文档
documents = [doc for page in pages for doc in unprocessed_documents_dict[page - 1] if page - 1 in unprocessed_documents_dict]

print(documents)
print(len(documents))

[Document(page_content=' \n \nKT-820Ti 数控车床系统 \n \n用 户 手 册 V2.0  \n \n \n南京开通自动化技术有限公司  820 \n830 ', metadata={'source': 'merge.pdf', 'page': 0}), Document(page_content=' \n \n \n技术支持：18913395570 / 02 5-87187350 转 8020  \n浙江台州办：  \n地址：玉环市玉城街道环岛北路青马幼儿园旁边 \n售后：13301582740 \n浙江诸暨办：  \n地址：浙江省诸暨市店口镇万通路 78 号      \n售后：13305159747 \n浙江宁波办：  \n地址：慈溪市孙塘北路人和家园 2 号楼 3 单元 1911   \n售后：13305193446 \n华北办：  \n地址：山东省德州市德城区兴河 湾C区5号 楼1单 元 1033 室 \n售后：13305193947 \n河南办：  \n地址：河南省长葛市八七路金帝苑小区中排东楼 602 室 \n售后：13301587698 \n江苏办：  \n地址：江苏省常州市武进区丁堰街道芳渚小区 298 号龙腾大酒店旁 \n售后：13305140584 \n \n \n南京开通自动化技术有限公司  \nNanjing KaiTong Automation Technology Co.,Ltd \n \n地址 : 南京市江宁区福英路1001 号-34栋 \n（联东U 谷.南京国际企业港） \n电话： +86 025 87187350 \n传真： +86 025 87187351                                \n网址： http://www.ktcnc.com \nEmail: ktcnc@ktcnc.com 数控系统使用手册  \n微信公众号  ', metadata={'source': 'merge.pdf', 'page': 1}), Document(page_content=' 1  \n第一篇  编程篇\nKT-820TI数控车床系统  ..................................................................

In [254]:
from langchain.text_splitter import CharacterTextSplitter
os.environ["OPENAI_API_KEY"] = key_pool[3]


text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever()

from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0613"), chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

prompt = f"""
    下面是用户提的问题,以'''为分割符号
    '''
    {question}
    '''
    如果问题是数学表达式，不用参考文本，直接计算结果。
    回答要求语气温和有礼貌、并且回答内容要详细、有具体文本支撑观点。
    """

res = qa(prompt)
answer, docs = res['result'], res['source_documents']
end_time = time.time()
execution_time = end_time - start_time
print(f"代码运行时间: {execution_time} 秒")
print(answer)


代码运行时间: 134.86705374717712 秒
很抱歉，我无法理解您的问题。请提供更多细节或明确您的问题，我将尽力帮助您。
